In [1]:
#import packages
import pandas as pd
import requests
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import urllib.request, json
import requests
import urllib.parse
import datetime
import nltk
nltk.download([
        "names",
        "stopwords",
        "averaged_perceptron_tagger",
        "vader_lexicon",
        "punkt",])

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package names to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/lizjohnson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### NYC Open Data:
    
SODA API Urls:

* 311 Call Inquiries: `https://data.cityofnewyork.us/resource/wewp-mm3p.json`

 **Important Column Names:**

   * uniqueid = 'UNIQUE_ID'
   * cat_clnm = 'AGENCY_NAME'
   * subcat_clnm = 'INQUIRY_NAME'
   * subsubcat_clnm = 'BRIEF_DESCRIPTION'
   * datetime_clnm = 'DATE_TIME'


* 311 Web Inquiries: `https://data.cityofnewyork.us/resource/vwpc-kje2.json`

 **Important Column Names:**

   * uniqueid = 'simple_id'
   * cat_clnm = 'service_name'
   * subcat_clnm = 'brief_description'
   * subsubcat_clnm = 'detailed_description'
   * datetime_clnm = 'updated_date'

In [ ]:
calls =  'https://data.cityofnewyork.us/resource/wewp-mm3p.json'

uniqueid = 'UNIQUE_ID'
cat_clnm = 'AGENCY_NAME'
subcat_clnm = 'INQUIRY_NAME'
subsubcat_clnm = 'BRIEF_DESCRIPTION'
datetime_clnm = 'DATE_TIME'
startdatetime = '2019-01-01T00:00:00.000'
enddatetime =  '2020-01-01T00:00:00.000'

query = (calls +'?'
    "$select=count(*) as count,%s as cat, %s as subcat, %s as subsubcat"
    "&$where=%s between '"+startdatetime+"' and '"+enddatetime+"'"
    "&$group=cat,subcat,subsubcat"
    "&$order=count DESC"
    "&$limit=500000"
    )%(cat_clnm,subcat_clnm,subsubcat_clnm,datetime_clnm)
query = query.replace(" ", "%20")
print(query)
response = urllib.request.urlopen(query)
data = json.loads(response.read())

https://data.cityofnewyork.us/resource/wewp-mm3p.json?$select=count(*)%20as%20count,AGENCY_NAME%20as%20cat,%20INQUIRY_NAME%20as%20subcat,%20BRIEF_DESCRIPTION%20as%20subsubcat&$where=DATE_TIME%20between%20'2019-01-01T00:00:00.000'%20and%20'2020-01-01T00:00:00.000'&$group=cat,subcat,subsubcat&$order=count%20DESC&$limit=500000


In [ ]:
call_comps = pd.DataFrame(data, columns = data[0].keys())
call_comps.head(10)

In [ ]:
len(call_comps)

In [ ]:
len(call_comps[call_comps['cat'].isna()])

In [ ]:
web = 'https://data.cityofnewyork.us/resource/vwpc-kje2.json'

uniqueid = 'service_id'
cat_clnm = 'service_name'
subcat_clnm = 'brief_description'
subsubcat_clnm = 'detailed_description'
datetime_clnm = 'updated_date'
startdatetime = '2019-01-01T00:00:00.000'
enddatetime =  '2020-01-01T00:00:00.000'

query = (web +'?'
    "$select=count(*) as count,%s as cat, %s as subcat, %s as subsubcat"
    "&$where=%s between '"+startdatetime+"' and '"+enddatetime+"'"
    "&$group=cat,subcat,subsubcat"
    "&$order=count DESC"
    "&$limit=500000"
    )%(cat_clnm,subcat_clnm,subsubcat_clnm,datetime_clnm)
query = query.replace(" ", "%20")
print(query)
response = urllib.request.urlopen(query)
data = json.loads(response.read())

In [ ]:
web_comps = pd.DataFrame(data, columns = data[0].keys())
web_comps.head(10)

In [ ]:
all_comps = call_comps.append(web_comps)
all_comps.head()

In [ ]:
call_comps['combined_cats'] = call_comps['cat'].fillna('') \
                      .str.cat(call_comps['subcat'].fillna(''), sep=' ') \
                      .str.cat(call_comps['subsubcat'].fillna(''), sep=' ')

call_comps.head()

In [ ]:
#clean combined_cats column to prepare for NLTK processing
#lowercase
call_comps['combined_cats_lower'] = call_comps['combined_cats'].str.lower()
#make list
txt_ls = call_comps['combined_cats_lower'].tolist()

In [ ]:
import re, string

tmp_list_unique = []

for item in txt_ls:
    
    item = item.replace('sanitation', 'sanitation waste')
    item = item.replace('parks', 'parks recreation')
    item = item.replace('recreation', 'parks recreation')
    item = item.replace('watr', 'water')
    item = item.replace('wats', 'water')
    item = item.replace('wrs', 'waste')
    item = item.replace('<p>', ' ')
    item = item.replace('<div>', ' ')
    
    item = item.replace('park maintenance', 'recreation and parks maintenance')
    item = item.replace('park rules', 'recreation and parks rules')
    item = item.replace('park equipment', 'recreation and parks equipment')
    item = item.replace('park lawn', 'recreation and parks lawn')
    item = item.replace('park programs', 'recreation and parks program')
    item = item.replace('park light', 'recreation and parks light')
    item = item.replace('park rangers', 'recreation and parks rangers')
    item = item.replace('park surface', 'recreation and parks surface')
    item = item.replace('rec and park', 'recreation and parks')
    
    item = item.replace('wildlife', 'wildlife animal')
    item = item.replace('pigeon', 'pigeon animal')
    item = item.replace('bats', 'bats animal')
    item = item.replace('dog', 'dog animal')
    item = item.replace('stray', 'stray animal')
    item = item.replace('coyote', 'coyote animal')
    item = item.replace('pot hole', 'pothole street')
    item = item.replace('sinkhole', 'sinkhole pothole street')
    
    item = item.replace('parking', 'parking vehicles')
    item = item.replace('bugs', 'public health bugs pest control')
    itme = item.replace('mosquitoes', 'public health mosquito pest control')
    itme = item.replace('mosquito', 'public health mosquito pest control')
    itme = item.replace('roaches', 'public health roaches pest control')
    item = item.replace('mice', 'public health rats rodent pest control')
    item = item.replace('rodent', 'public health rodent rats pest control')
    item = item.replace('dumping', 'dumping waste')
    item = item.replace('illdump', 'illegal dumping waste')
    item = item.replace('plmb', 'plumbming')
    item = item.replace('haz', 'hazard')
    item = item.replace('cons', 'construction')
    item = item.replace('prmt', 'permit')
    item = item.replace('cnflct', 'conflict')
    item = item.replace('insp', 'inspection')
    item = item.replace('litter', 'litter waste')
    item = item.replace('isd', '')
    item = item.replace('traum counseling', 'trauma counseling')
    item = item.replace('dirty', 'dirty unsanitary')
    item = item.replace('bldgmaint', 'building maintenance')
    item = item.replace('derelict', 'abandonded')
    item = item.replace('blighted', 'abandonded')
    item = item.replace('vacant', 'abandonded')
    item = item.replace('building code', 'inspection code enforcement violation')
    item = item.replace('inspection', 'inspection code enforcement violation')
    item = item.replace('code enforcement', 'inspection code enforcement violation')
    item = item.replace('code violations', 'inspection code enforcement violation')
    item = item.replace('housing violations', 'inspection code enforcement violation')
    item = item.replace('property violations', 'inspection code enforcement violation')
    item = item.replace('codes compliance', 'inspection code enforcement violation')
    item = item.replace('transit', 'transit transportation')
    item = item.replace('transportation', 'transit transportation')
    item = item.replace('tenant', 'tenant housing rental')
    item = item.replace('rental issues', 'tenant housing rental')
    item = item.replace('brush', 'brush debris bulky')
    item = item.replace('loose leaf', 'brush debris bulky')
    item = item.replace('debris', 'brush debris bulky')
    item = item.replace('stlight(s)', 'street light')
    item = item.replace('collectn', 'bulky waste collection')
    item = item.replace('household appliances', 'bulky waste collection')
    item = item.replace('special collections', 'bulky waste collection')
    item = item.replace('property maintenance', 'property maintenance building residential')
    item = item.replace('building residential', 'property maintenance building residential')
    item = item.replace('forestry', 'tree')
    item = item.replace('national grid', 'electricity gas')
    item = item.replace('bicycle', 'bike')
    item = item.replace('botanical', 'tree')
    item = item.replace('property', 'property building')
    item = item.replace('pay station', 'pay station parking meter')
    item = item.replace('general inquiry', 'general information')
    item = item.replace('question', 'general information')
    item = item.replace('general questions', 'general information')
    item = item.replace('general request', 'general information')
    item = item.replace('literature request', 'general information')
    item = item.replace('program information', 'general information')
    item = item.replace('information request', 'general information')
    item = item.replace('records request', 'general information')
    
    item = item.replace('manhole', 'manhole drain')
    item = item.replace('missed collection', 'missed collection waste')
    item = item.replace('roadway', 'roads')
    
    item = item.replace('tax - ', 'tax taxpayer treasury')
    item = item.replace('compliment', 'commendation')
    
    item = item.replace('constructiontruction', 'construction')
    item = item.replace('contructionumber', 'construction')
    
    for number in ['1','2','3','4','5','6','7','8','9','0']:    
        item = item.replace(number,'')
    
    #remove symbols exc NEW CODE INSERTED BY LIZ JOHNSON 11/15/22
    symbols = "!\"\'#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in symbols:
        item = np.char.replace(item, i, '')
    
    tmp_list_unique.append(str(item).strip())

In [ ]:
print(tmp_list_unique[0:10])

In [ ]:
stopwords = nltk.corpus.stopwords.words('english')
#remove stopwords
tmp_list_unique = [[word for word in sentence.split() if word.lower() not in stopwords] for sentence in tmp_list_unique]
#join split sentences back
tmp_list_unique = [" ".join(split_sentence) for split_sentence in tmp_list_unique]

In [ ]:
print(tmp_list_unique[0:10])

In [ ]:
call_comps['processed_comps'] = tmp_list_unique

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

# Initialize the sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

# Define a function to calculate sentiment scores for each row in the dataframe
def get_sentiment_scores(row):
    # Use the VADER sentiment analyzer to get a sentiment score for the combined categories text
    sentiment_scores = analyzer.polarity_scores(row['processed_comps'])
    # Return the sentiment score dictionary
    return sentiment_scores

# Apply the function to each row in the dataframe and store the results in a new column
call_comps['sentiment_scores'] = call_comps.apply(get_sentiment_scores, axis=1)


In [ ]:
call_comps.head()

In [ ]:
call_comps[['neg','neu', 'pos', 'compound']] = call_comps['sentiment_scores'].apply(pd.Series)
call_comps.head()

In [ ]:
# Calculate the mean compound sentiment score for each category
mean_sentiment_scores = call_comps.groupby('combined_cats')['sentiment_scores'].apply(lambda x: x.apply(lambda y: y['compound']).mean())

In [ ]:
#save outputs
call_comps.to_csv('../Outputs/311calls_complaints_vadar_results.csv')

### Let's try and visualize the 311 inquiries' sentiment analysis results...

# RESTART FROM HERE

In [ ]:
call_comps = pd.read_csv('../Outputs/311calls_complaints_vadar_results.csv', index_col = 0)
call_comps.head()

In [ ]:
mean_values = call_comps[['cat','neg','neu','pos', 'compound']].groupby('cat').agg({'neg': 'mean', 'neu': 'mean', 'pos': 'mean', 'compound': 'mean'})
median_values = call_comps[['cat','neg','neu','pos', 'compound']].groupby('cat').agg({'neg': 'median', 'neu': 'median', 'pos': 'median', 'compound': 'median'})

In [ ]:
mean_values.sort_values(by = 'compound', ascending = False).head(10)

In [ ]:
median_values.sort_values(by = 'compound', ascending = False).head(10)

In [ ]:
mean_values.sort_values(by = 'compound').head(10)

In [ ]:
median_values.sort_values(by = 'compound').head(10)

In [ ]:
top_5_pos = list(mean_values.sort_values(by='compound', ascending=False)['compound'].head(5).index)
top_5_neg = list(mean_values.sort_values(by='compound', ascending=False)['compound'].tail(5).index)
print(top_5_pos,top_5_neg)

In [ ]:
len(call_comps[call_comps['cat'] == top_5_neg[-1]])

In [ ]:
call_comps[call_comps['cat'] == top_5_neg[-1]].sort_values(by = 'compound')

In [ ]:
len(call_comps[call_comps['cat'].isin(top_5_pos[0:2])])

In [ ]:
call_comps[call_comps['cat'].isin(top_5_pos[0:2])].sort_values(by = 'compound', ascending = False)

In [ ]:
plt.hist(call_comps['compound'])
plt.show()

In [ ]:
#ten most positive complaints
pos_10 = call_comps.sort_values(by='compound', ascending=False).head(10)
#ten most negative complaints
neg_10 = call_comps.sort_values(by='compound', ascending=False).tail(10)

In [ ]:
import textwrap

neg_scores = pos_10['neg'][::-1]
neu_scores = pos_10['neu'][::-1]
pos_scores = pos_10['pos'][::-1]
comps_scores = pos_10['compound'][::-1]

fig, ax = plt.subplots(figsize = (8,8))
ax.barh(np.arange(10), pos_scores, label='Positive', color='green')
ax.barh(np.arange(10), neu_scores, left=pos_scores, label='Neutral', color='gray')
ax.barh(np.arange(10), neg_scores, left=pos_scores+neu_scores, label='Negative', color='red')
ax.set_xlabel('Sentiment Score')
ax.set_title('Top 10 Positive Inquiries by Compound Score')
ax.set_yticks(np.arange(10))
# Wrap the text of each y-tick label onto multiple lines
wrapped_labels = [textwrap.fill(label, width=75) for label in pos_10['combined_cats'][::-1]]
ax.set_yticklabels(wrapped_labels, fontsize = 8)
# Add label on top of each bar
for i, score in enumerate(comps_scores):
    ax.text(score+0.01, i, f'{score:.2f}', ha='left', va='center')
legend = ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0))
plt.savefig('../Outputs/Top_10_Positive_Compound_barh.png', bbox_inches='tight')
plt.show()

In [ ]:
neg_scores = neg_10['neg']
neu_scores = neg_10['neu']
pos_scores = neg_10['pos']
comps_scores = neg_10['compound']

fig, ax = plt.subplots(figsize = (8,8))
ax.barh(np.arange(10), pos_scores, label='Positive', color='green')
ax.barh(np.arange(10), neu_scores, left=pos_scores, label='Neutral', color='gray')
ax.barh(np.arange(10), neg_scores, left=pos_scores+neu_scores, label='Negative', color='red')
ax.set_xlabel('Sentiment Score')
ax.set_title('Top 10 Negitive Inquiries by Compound Score')
ax.set_yticks(np.arange(10))
# Wrap the text of each y-tick label onto multiple lines
wrapped_labels = [textwrap.fill(label, width=75) for label in neg_10['combined_cats']]
ax.set_yticklabels(wrapped_labels, fontsize = 8)
# Add label on top of each bar
for i, score in enumerate(comps_scores):
    ax.text(abs(score)+0.01, i, f'{score:.2f}', ha='right', va='center')
legend = ax.legend(loc='upper right', bbox_to_anchor=(1.2, 1.0))
plt.savefig('../Outputs/Top_10_Negative_Compound_barh.png', bbox_inches='tight')
plt.show()

In [ ]:
#look into neutrals
len(call_comps[call_comps['compound'] == 0])

In [ ]:
len(call_comps[call_comps['compound'] != 0])

In [ ]:
neu = call_comps[call_comps['compound'] == 0]
neu['cat'].value_counts()

In [ ]:
neu.head(15)

In [ ]:
len(neu[neu['neu'] != 1])

In [ ]:
call_comps[call_comps['combined_cats_lower'].str.contains('homeless')].sort_values(by = 'compound').head(15)

In [ ]:
call_comps[call_comps['combined_cats_lower'].str.contains('noise')].sort_values(by = 'compound').head(15)

In [ ]:
call_comps[call_comps['combined_cats_lower'].str.contains('graffiti')].sort_values(by = 'compound').head(15)